<a href="https://colab.research.google.com/github/aayushh04/dlt/blob/main/week_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Import libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow_datasets as tfds # Import tensorflow_datasets

# Step 2: Load the pretrained VGG16 model without top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Step 3: Freeze all pretrained layers to use as a fixed feature extractor
for layer in base_model.layers:
    layer.trainable = False

# Step 4: Add custom classification layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)  # Example: binary classification

# Step 5: Combine base model and custom head into a single model
model = Model(inputs=base_model.input, outputs=predictions)

# Step 6: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 7: Load a prebuilt dataset using TensorFlow Datasets
# Using the Cats vs Dogs dataset as an example
print("Loading Cats vs Dogs dataset...")
# Ensure the dataset is downloaded and prepared correctly
try:
    (ds_train, ds_val), ds_info = tfds.load(
        'cats_vs_dogs',
        split=['train[:80%]', 'train[80%:]'], # Split data into 80% train, 20% validation
        with_info=True,
        as_supervised=True # Load as (image, label) tuples
    )
    print("Dataset loaded successfully.")
except Exception as e:
    print(f"Error loading dataset: {e}")
    # Exit or handle the error appropriately if the dataset cannot be loaded
    exit()


# Step 7.1: Preprocess the dataset
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = ds_info.features['label'].num_classes # Get the number of classes from dataset info

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.keras.applications.vgg16.preprocess_input(image) # VGG16 specific preprocessing
    label = tf.one_hot(label, NUM_CLASSES) # One-hot encode the labels
    return image, label

# Apply preprocessing and batching to the datasets
train_generator = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_generator = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# Step 8: Train only the custom classifier layers
print("\nStarting model training...")
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)
print("Model training completed.")

# Step 9: Evaluate model performance
print("\nEvaluating model performance...")
loss, accuracy = model.evaluate(val_generator)
print(f"\nValidation Accuracy: {accuracy * 100:.2f}%")

# Step 10: (Optional) Save the trained model
model.save('vgg16_feature_extractor_model.h5')

print("\nModel training completed and saved successfully!")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Loading Cats vs Dogs dataset...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.XOWP19_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
Dataset loaded successfully.

Starting model training...
Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 165s 255ms/step - accuracy: 0.9483 - loss: 2.6119 - val_accuracy: 0.9828 - val_loss: 0.0976
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 131s 225ms/step - accuracy: 0.9828 - loss: 0.0871 - val_accuracy: 0.9824 - val_loss: 0.0841
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 132s 226ms/step - accuracy: 0.9855 - loss: 0.0626 - val_accuracy: 0.9828 - val_loss: 0.0905
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 132s 226ms/step - accuracy: 0.9871 - loss: 0.0587 - val_accuracy: 0.9845 - val_loss: 0.0756
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 132s 226ms/step - accuracy: 0.9879 - loss: 0.0581 - val_accuracy: 0.9830 - val_loss: 0.1015
Model training completed.

Evaluating model performance...
146/146 ━━━━━━━━━━━━━━━━━━━━ 26s 179ms/step - accuracy: 0.9825 - loss: 0.0886



Validation Accuracy: 98.30%

Model training completed and saved successfully!
